<a href="https://colab.research.google.com/github/ragaashritha/2015/blob/master/Raga_Session_5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF #28,26,3 
        self.batchnorm1 = nn.BatchNorm2d(8)
        #self.dp1 = nn.Dropout(0.20)

        self.conv2 = nn.Conv2d(8, 20, 3)#26,24,5
        self.batchnorm2 = nn.BatchNorm2d(20)
        self.dp2 = nn.Dropout(0.10)

        self.pool1 = nn.MaxPool2d(2, 2)#24,12,10

        self.conv3 = nn.Conv2d(20, 10, 1)#12,12,10
        self.batchnorm3 = nn.BatchNorm2d(10)
        self.dp3 = nn.Dropout(0.10)

        self.conv4 = nn.Conv2d(10, 14, 3)#12,10,12
        self.batchnorm4 = nn.BatchNorm2d(14)
        self.dp4 = nn.Dropout(0.10)

        #self.pool2 = nn.MaxPool2d(2, 2)#10,5,24
        
        self.conv5 = nn.Conv2d(14, 18, 3)#10,8,12
        self.batchnorm5 = nn.BatchNorm2d(18)
        self.dp5 = nn.Dropout(0.10)

        self.conv6 = nn.Conv2d(18, 24, 3)#8,6,14
        self.batchnorm6 = nn.BatchNorm2d(24)
        self.dp6 = nn.Dropout(0.10)

    
        self.conv7_avgp = nn.AvgPool2d(kernel_size=6)
        self.conv8 = nn.Conv2d(24, 10, 1)#6,4,18

    def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      x = self.batchnorm1(x)

      x = self.conv2(x)
      x = F.relu(x)
      x = self.batchnorm2(x)
      x = self.dp2(x)

      x = self.pool1(x)

      x = self.conv3(x)
      x = F.relu(x)
      x = self.batchnorm3(x)
      x = self.dp3(x)

      x = self.conv4(x)
      x = F.relu(x)
      x = self.batchnorm4(x)
      x = self.dp4(x)

      x = self.conv5(x)
      x = F.relu(x)
      x = self.batchnorm5(x)
      x = self.dp5(x)

      x = self.conv6(x)
      x = F.relu(x)
      x = self.batchnorm6(x)
      x = self.dp6(x)

     
      x = self.conv7_avgp(x)
      x = self.conv8(x) 
        
      x = x.view(-1, 10)
      return F.log_softmax(x)

In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 20, 24, 24]           1,460
       BatchNorm2d-4           [-1, 20, 24, 24]              40
           Dropout-5           [-1, 20, 24, 24]               0
         MaxPool2d-6           [-1, 20, 12, 12]               0
            Conv2d-7           [-1, 10, 12, 12]             210
       BatchNorm2d-8           [-1, 10, 12, 12]              20
           Dropout-9           [-1, 10, 12, 12]               0
           Conv2d-10           [-1, 14, 10, 10]           1,274
      BatchNorm2d-11           [-1, 14, 10, 10]              28
          Dropout-12           [-1, 14, 10, 10]               0
           Conv2d-13             [-1, 18, 8, 8]           2,286
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-8.0, 8.0), fill=(1,)),                       
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
test_loss_ = []
train_loss = 0

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_correct = 0
    pbar = tqdm(train_loader,leave=False,position=0)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_loss_.append(test_loss)


    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.016, momentum=0.9)
scheduler = StepLR(optimizer, step_size=2, gamma=0.85)

In [7]:
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
   # scheduler.step()
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
                                                                                        

Epoch: 1,LR: 0.016.
Train set: train Average loss: 0.0000, train_Accuracy: 53124/60000 (88.5400%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0708, Accuracy: 9804/10000 (98.0400%)



Epoch: 2,LR: 0.016.
Train set: train Average loss: 0.0000, train_Accuracy: 58430/60000 (97.3833%)



...

Epoch: 18,LR: 0.016.
Train set: train Average loss: 0.0000, train_Accuracy: 59382/60000 (98.9700%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0218, Accuracy: 9937/10000 (99.3700%)



Epoch: 19,LR: 0.016.
Train set: train Average loss: 0.0000, train_Accuracy: 59428/60000 (99.0467%)




Test set: Average loss: 0.0236, Accuracy: 9932/10000 (99.3200%)



# New Section

In [ ]:
# Specify a path
PATH = "entire_model.pt"

# Save
torch.save(net, PATH)

# Load
model = torch.load(PATH)
model.eval()

['raga', 'is', 'a', 'good', 'girl']